# Projeto Aplicado III - Construindo um sistema de recomendação

O projeto tem como objetivo **desenvolver um sistema de recomendação de filmes** utilizando o *5000 Movie Dataset*, disponível no Kaggle, que reúne informações detalhadas sobre cerca de 5.000 produções do The Movie Database (TMDb). Esse conjunto de dados inclui variáveis como **orçamento, gêneros, popularidade, empresas produtoras, países de produção, elenco e equipe técnica**, permitindo a aplicação de técnicas de aprendizado de máquina para sugerir conteúdos mais relevantes aos usuários.

A iniciativa busca não apenas aprimorar competências práticas em **ciência de dados e mineração de dados**, mas também contribuir para os **Objetivos de Desenvolvimento Sustentável (ODS)** da ONU, como o **ODS 9 (Inovação e Infraestrutura)**, o **ODS 4 (Educação de Qualidade)** e o **ODS 10 (Redução das Desigualdades)**. Assim, o sistema pretende oferecer recomendações personalizadas que promovam maior **diversidade cultural e inclusão digital**


Para a análise e desenvolvimento deste projeto, será necessário utilizar um conjunto de bibliotecas do ecossistema Python, cada uma com um papel específico no fluxo de trabalho de **análise exploratória** e **construção do sistema de recomendação**.

- **pandas**: essencial para manipulação e análise de dados tabulares, permitindo leitura, limpeza e transformação dos datasets `tmdb_5000_movies` e `tmdb_5000_credits`.  
- **numpy**: fornece suporte para operações matemáticas e vetorização, aumentando a eficiência no processamento de dados.  
- **matplotlib**: utilizada para criar visualizações básicas, como gráficos de barras, dispersão e histogramas.  
- **seaborn**: complementa o matplotlib oferecendo visualizações estatísticas mais sofisticadas e com estética aprimorada.  
- **scikit-learn**: importante para o pré-processamento dos dados, cálculo de métricas de avaliação e implementação de algoritmos de recomendação baseados em aprendizado de máquina.  
- **scipy**: será usada para cálculos matemáticos 
- **surprise (scikit-surprise)**: biblioteca especializada em sistemas de recomendação, especialmente nos modelos colaborativos como **SVD** e **KNNBasic**, permitindo comparar técnicas.  
- **lightfm**: possibilita a construção de sistemas de recomendação híbridos, combinando informações de conteúdo e interações de usuários.  
- **tensorflow / pytorch**: úteis caso seja necessário evoluir para modelos de recomendação mais avançados baseados em **deep learning**.


Abaixo, será realizada a importação das bibliotecas necessárias para a **análise exploratória** e a **construção do sistema de recomendação**.

Antes de iniciar a análise, é necessário garantir que todas as bibliotecas utilizadas neste projeto estejam instaladas no ambiente.  
Para isso, basta executar o comando abaixo no terminal ou em uma célula do Jupyter Notebook (prefixado com `!`):

```bash
pip install -r requirements.txt


In [ ]:
# Bibliotecas para análise de dados
import pandas as pd
import numpy as np
import ast

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas para sistemas de recomendação
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse




In [71]:
# Leitura dos datasets
movies_df = pd.read_csv("datasets/tmdb_5000_movies.csv")
credits_df = pd.read_csv("datasets/tmdb_5000_credits.csv")

# Mostrando as primeiras linhas dos datasets
print("📌 Movies Dataset:")
display(movies_df.head())

print("\n📌 Credits Dataset:")
display(credits_df.head())

# Informações básicas dos datasets
print("\n🔎 Informações do Movies Dataset:")
print(movies_df.info())

print("\n🔎 Informações do Credits Dataset:")
print(credits_df.info())


📌 Movies Dataset:


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124



📌 Credits Dataset:


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."



🔎 Informações do Movies Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   

Nesta etapa, será realizada a **seleção das colunas relevantes** para a construção do sistema de recomendação.  

O objetivo é reduzir o dataframe apenas às variáveis que realmente serão utilizadas no modelo, garantindo maior **eficiência no processamento**, evitando redundâncias e mantendo o foco nos atributos mais informativos para a geração das recomendações.

Além disso, também serão tratados os valores nulos para diminuir o ruído presente no dataset.

Também será feito um **merge com o dataframe de créditos**, permitindo integrar informações sobre elenco e equipe técnica ao conjunto de dados principal, enriquecendo assim a base para a construção do sistema de recomendação.


In [72]:
# Movies Dataset

# Filtrando somente linhas em que o filme está com status 'Released'
# Utilizamos o reset_index(drop=True) para resetar os índices do DataFrame após o filtro
movies_df = movies_df[movies_df['status'] == 'Released'].reset_index(drop=True)
# Agora, filtraremos apenas as colunas que nos interessam para a análise
movies_df = movies_df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'release_date', 'overview', 'runtime', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']]
# Tratando linhas com valores nulos
movies_df = movies_df.dropna().reset_index(drop=True)
# Convertendo a coluna 'release_date' para o tipo datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

# Credits Dataset
# Selecionando apenas as colunas que nos interessam
credits_df = credits_df[['movie_id', 'cast', 'crew']]
# Tratando linhas com valores nulos
credits_df = credits_df.dropna().reset_index(drop=True)
# Renomeando a coluna 'movie_id' para 'id' para facilitar o merge
credits_df = credits_df.rename(columns={'movie_id': 'id'})

# Merge dos datasets
# Realizando o merge dos datasets 'movies_df' e 'credits_df' com base na coluna 'id'
movies_df = movies_df.merge(credits_df, on='id')
# Mostrando as primeiras linhas do dataset final
print("\n📌 Dataset Final após Merge:")
display(movies_df)



📌 Dataset Final após Merge:


,id,title,genres,vote_average,vote_count,popularity,release_date,overview,runtime,keywords,production_companies,production_countries,spoken_languages,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,2009-12-10,"In the 22nd century, a paraplegic Marine is di...",162.0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,2007-05-19,"Captain Barbossa, long believed to be dead, ha...",169.0,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,2015-10-26,A cryptic message from Bond’s past sends him o...,148.0,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...","[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,2012-07-16,Following the death of District Attorney Harve...,165.0,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,2012-03-07,"John Carter is a war-weary, former military ca...",132.0,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4786,9367,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",6.6,238,14.269792,1992-09-04,El Mariachi just wants to play his guitar and ...,81.0,"[{""id"": 5616, ""name"": ""united states\u2013mexi...","[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...","[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4787,72766,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",5.9,5,0.642552,2011-12-26,A newlywed couple's honeymoon is upended by th...,85.0,[],[],[],[],"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4788,231617,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",7.0,6,1.444476,2013-10-13,"""Signed, Sealed, Delivered"" introduces a dedic...",120.0,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{"

Neste trecho de código é realizado o **tratamento das colunas no formato JSON** presentes nos datasets.  
Foram criadas funções auxiliares para converter as colunas que armazenam listas de dicionários em **listas de valores extraídos**, de forma a facilitar a manipulação e análise.  

- A função `parse_json_column` é responsável por percorrer colunas no formato JSON e **extrair os valores de um campo específico** (como `name`, `iso_3166_1` ou `iso_639_1`).  
- A função `extract_director` percorre a coluna `crew` e **identifica o diretor principal** de cada filme, criando uma nova coluna `director`.  

Após o processamento, as colunas originais em JSON são transformadas em listas de valores mais simples, e a coluna `crew` foi descartada, visto que sua informação relevante (o diretor) já foi extraída.  
Esse tratamento torna o dataframe mais **limpo e estruturado**, possibilitando seu uso na análise exploratória e na construção do sistema de recomendação.

In [73]:
# Funções para processar as colunas JSON
def parse_json_column(movies_df, column_name, key='name'):
    def extract_names(json_str):
        if isinstance(json_str, str):
            try:
                list_of_dicts = ast.literal_eval(json_str)
                if isinstance(list_of_dicts, list):
                    return [d[key] for d in list_of_dicts if key in d]
            except (ValueError, SyntaxError):
                pass
        return []
    movies_df[column_name] = movies_df[column_name].apply(extract_names)
    return movies_df

# Método para extrair o diretor do elenco
def extract_director(crew_json):
    if isinstance(crew_json, str):
        try:
            crew_list = ast.literal_eval(crew_json)
            for member in crew_list:
                if member.get('job') == 'Director':
                    return member.get('name')
        except (ValueError, SyntaxError):
            pass
    return np.nan

# Aplicando as funções para explodir as colunas JSON
movies_df = parse_json_column(movies_df, 'genres')
movies_df = parse_json_column(movies_df, 'keywords')
movies_df = parse_json_column(movies_df, 'production_companies')
movies_df = parse_json_column(movies_df, 'production_countries', key='iso_3166_1') 
movies_df = parse_json_column(movies_df, 'spoken_languages', key='iso_639_1')
movies_df = parse_json_column(movies_df, 'cast', key='name')
movies_df['director'] = movies_df['crew'].apply(extract_director)

# Removendo a coluna 'crew' original, pois já extraímos o diretor
movies_df = movies_df.drop(columns=['crew'])

# Mostrando as primeiras linhas do dataset após o tratamento das colunas JSON
print("\n📌 Dataset Final após tratamento de JSON:")
display(movies_df.head())


📌 Dataset Final após tratamento de JSON:


,id,title,genres,vote_average,vote_count,popularity,release_date,overview,runtime,keywords,production_companies,production_countries,spoken_languages,cast,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,2009-12-10,"In the 22nd century, a paraplegic Marine is di...",162.0,"[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[US, GB]","[en, es]","[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,2007-05-19,"Captain Barbossa, long believed to be dead, ha...",169.0,"[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...",[US],[en],"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,2015-10-26,A cryptic message from Bond’s past sends him o...,148.0,"[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[GB, US]","[fr, en, es, it, de]","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,2012-07-16,Following the death of District Attorney Harve...,165.0,"[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...",[US],[en],"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,2012-03-07,"John Carter is a war-weary, former military ca...",132.0,"[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],[US],[en],"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton


Abaixo, salvamos o *dataset* normalizado em um arquivo CSV, para que possamos reutilizá-lo novamente sem a necessidade de executar os métodos acima.

In [74]:
movies_df.to_csv("datasets/movies_cleaned.csv", index=False)

O processo de treinamento do **sistema de recomendação baseada em conteúdo** consiste em transformar as **informações textuais dos filmes** em representações numéricas que permitam medir similaridade. Para isso, os atributos mais relevantes — como **sinopse (overview), gêneros, palavras-chave, elenco e diretor** — são **concatenados em um único campo de texto**. Em seguida, aplica-se a técnica **TF-IDF (Term Frequency – Inverse Document Frequency)**, que gera uma matriz esparsa onde cada filme é representado por um vetor numérico que valoriza **termos mais característicos** e reduz a importância de **palavras comuns**.  

Com a matriz **TF-IDF** construída, o próximo passo é calcular a **similaridade do cosseno** entre os vetores, o que permite identificar filmes **mais próximos em termos de conteúdo**. Assim, dado um título escolhido pelo usuário, o sistema compara seu vetor com todos os outros e retorna uma **lista dos filmes mais semelhantes**. Esse processo **dispensa dados explícitos de usuários** e permite **recomendações personalizadas** a partir das características intrínsecas dos filmes.  

Além disso, também foi criado um método que dado um texto genérico de busca (em inglês), o sistema compara o vetor do texto genérico com o conteúdo do filme baseado nas colunas do *dataset* normalizado.


In [75]:
# Funções auxiliares para preparação dos dados
def normalize_token(tok: str) -> str:
    """Normaliza um token: minúsculas, espaços por underscores, remove caracteres especiais."""
    t = tok.lower().strip()
    t = re.sub(r"\s+", "_", t)
    t = re.sub(r"[^a-z0-9_\-]", "", t)
    return t

def parse_listish(x):
    """Converte uma string que representa uma lista em uma lista de tokens normalizados."""
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        s = x.strip()
        if s.startswith('[') and s.endswith(']'):
            try:
                val = ast.literal_eval(s)
                if isinstance(val, list):
                    return [normalize_token(str(v)) for v in val if str(v).strip()]
            except Exception:
                pass
        return [normalize_token(s)] if s else []
    return []

def clean_text(s: str) -> str:
    """Limpa um texto: minúsculas, remove caracteres especiais, múltiplos espaços."""
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def join_tokens(tokens):
    """Une uma lista de tokens em uma string, ignorando valores não-string ou vazios."""
    return " ".join([t for t in tokens if isinstance(t, str) and t])

def prepare_data(csv_path="datasets/movies_cleaned.csv"):
    """Prepara os dados para o sistema de recomendação."""
    df = pd.read_csv(csv_path)

    # listas normalizadas
    df["director_list"] = df["director"].apply(lambda x: [normalize_token(x)] if isinstance(x, str) and x.strip() else [])
    for col in ["genres", "keywords", "cast"]:
        df[col + "_list"] = df[col].apply(parse_listish)

    # texto livre
    df["overview_clean"] = df["overview"].apply(clean_text)

    # bolsa simbólica (tokens)
    df["symbolic_bag"] = (
        df["genres_list"].apply(join_tokens) + " " +
        df["keywords_list"].apply(join_tokens) + " " +
        df["cast_list"].apply(join_tokens) + " " +
        df["director_list"].apply(join_tokens)
    ).str.strip()

    # campo final
    df["final_text"] = (df["overview_clean"] + " " + df["symbolic_bag"]).str.strip()

    # guardamos algumas colunas úteis para posterior ranking
    base_cols = ["id", "title", "vote_average", "vote_count", "popularity"]
    keep = [c for c in base_cols if c in df.columns]
    idx_df = df[keep].copy()
    return df, idx_df

def train_or_load(df, vec_path="datasets/tfidf_vectorizer.pkl", mat_path="datasets/tfidf_matrix.npz", idx_path="datasets/movies_index.csv"):
    """Treina ou carrega o modelo TF-IDF e a matriz esparsa."""
    if os.path.exists(vec_path) and os.path.exists(mat_path) and os.path.exists(idx_path):
        vectorizer = joblib.load(vec_path)
        tfidf_matrix = sparse.load_npz(mat_path)
        idx_df = pd.read_csv(idx_path)
        return vectorizer, tfidf_matrix, idx_df

In [76]:
title_to_index = {t.lower(): i for i, t in enumerate(movies_df["title"].astype(str))}
vectorizer, tfidf_matrix, idx_df = train_or_load(prepare_data())

In [ ]:
def recommend_movies(query_title: str, top_n: int = 10, exclude_same: bool = True):
    """Recomenda filmes similares a um título dado."""
    idx = title_to_index.get(query_title.lower())
    if idx is None:
        raise ValueError(f"Título não encontrado no dataset: {query_title}")
    sims = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).ravel()
    order = np.argsort(-sims)
    results = []
    for i in order:
        if exclude_same and i == idx:
            continue
        results.append((movies_df.at[i, "title"], float(sims[i]), int(movies_df.at[i, "vote_count"]), float(movies_df.at[i, "vote_average"])))
        if len(results) >= top_n:
            break
    rec_df = pd.DataFrame(results, columns=["title", "similarity", "vote_count", "vote_average"])
    return rec_df

In [78]:
def recommend_by_text(query_text: str, top_n: int = 10, rerank_popularity: bool = True):
    """
    Gera recomendações a partir de um texto livre.
    - query_text: ex. "space adventure with aliens and strong female lead"
    - rerank_popularity: se True, reordena suavemente por popularidade (evita filmes obscuros no topo)
    """
    assert isinstance(query_text, str) and query_text.strip(), "query_text deve ser uma string não vazia."
    # carregamento/treino (idempotente)
    df, idx_df = prepare_data("datasets/movies_cleaned.csv")
    vectorizer, tfidf_matrix, _ = train_or_load(df)

    # Vetoriza a consulta usando o vocabulário TF-IDF treinado
    q = clean_text(query_text)
    q_vec = vectorizer.transform([q])

    # Similaridade com todos os filmes
    sims = cosine_similarity(q_vec, tfidf_matrix).ravel()

    # Monta dataframe de resultados
    out = idx_df.copy()
    out["similarity"] = sims

    # (Opcional) re-rank por popularidade/nota com mistura linear simples
    # Normaliza popularidade e vote_average para [0,1]
    if rerank_popularity:
        if "popularity" in out.columns:
            pop = out["popularity"].astype(float)
            pop_norm = (pop - pop.min()) / (pop.max() - pop.min() + 1e-9)
        else:
            pop_norm = 0.0

        if "vote_average" in out.columns:
            va = out["vote_average"].astype(float)
            va_norm = (va - va.min()) / (va.max() - va.min() + 1e-9)
        else:
            va_norm = 0.0

        # mistura: 70% similaridade + 20% popularidade + 10% nota
        final_score = 0.7 * out["similarity"].values + 0.2 * pop_norm + 0.1 * va_norm
        out["final_score"] = final_score
        out = out.sort_values(by=["final_score", "similarity"], ascending=False)
    else:
        out = out.sort_values(by="similarity", ascending=False)

    cols_show = [c for c in ["title", "similarity", "final_score", "vote_average", "vote_count", "popularity", "id"] if c in out.columns]
    return out[cols_show].head(top_n).reset_index(drop=True)

In [87]:
sample_title = "The Dark Knight"
rec_demo = recommend_movies(sample_title, top_n=10, exclude_same=False)
query = "bat costume, dark city, vigilante, high tech gadgets"
recs = recommend_by_text(query, top_n=10, rerank_popularity=False)

print("Artefatos salvos em /mnt/data:")
print(" - tfidf_vectorizer.pkl")
print(" - tfidf_matrix.npz")
print(" - movies_index.csv")
print("\nFunção disponível: recommend_movies(<título>, top_n=10)")

Artefatos salvos em /mnt/data:
 - tfidf_vectorizer.pkl
 - tfidf_matrix.npz
 - movies_index.csv

Função disponível: recommend_movies(<título>, top_n=10)


In [ ]:
print(recs)

                       title  similarity  vote_average  vote_count  \
0                       Bats    0.161216           4.1          35   
1      Johnny English Reborn    0.096073           6.0        1007   
2             Batman Returns    0.091619           6.6        1673   
3  Gremlins 2: The New Batch    0.090849           6.2         652   
4        Hobo with a Shotgun    0.090224           5.7         211   
5           Charlie's Angels    0.079748           5.6        1232   
6         The Stepford Wives    0.079379           5.4         334   
7                     Sheena    0.077996           5.0          22   
8           Mr. & Mrs. Smith    0.071715           6.5        2965   
9                        RED    0.070529           6.6        2808   

   popularity     id  
0    1.537859  10496  
1   35.658500  58233  
2   59.113174    364  
3   33.986370    928  
4   16.900433  49010  
5   40.203950   4327  
6   19.224754   9890  
7    4.020194  24264  
8   44.635452    787  

In [90]:
print(rec_demo)

                                     title  similarity  vote_count  \
0                          The Dark Knight    1.000000       12002   
1                    The Dark Knight Rises    0.203810        9106   
2                           Batman Returns    0.174953        1673   
3                            Batman Begins    0.171536        7359   
4                           Batman Forever    0.168999        1498   
5                                   Batman    0.136889        2096   
6  Batman: The Dark Knight Returns, Part 2    0.105850         419   
7       Batman v Superman: Dawn of Justice    0.102948        7004   
8                           Batman & Robin    0.101061        1418   
9                      Law Abiding Citizen    0.090189        1486   

   vote_average  
0           8.2  
1           7.6  
2           6.6  
3           7.5  
4           5.2  
5           7.0  
6           7.9  
7           5.7  
8           4.2  
9           7.2  
